# Задача 1
На сайте запущен А/В тест с целью увеличить доход. В приложенном excel файле вы найдете сырые данные по результатам эксперимента – user_id, тип выборки variant_name и доход принесенный пользователем revenue. Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

## Загрузка библиотек и данных

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import warnings
warnings.filterwarnings('ignore')


file_name = './data/AB_Test_Results.xlsx'
xl_data = pd.ExcelFile(file_name)
df = xl_data.parse('AB Test Results')
df['REVENUE'] = df['REVENUE'].astype(float)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       10000 non-null  int64  
 1   VARIANT_NAME  10000 non-null  object 
 2   REVENUE       10000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB


Типы данных корректны. В строках отсутствуют нулевые значения (Null). Сведем данные по каждому пользователю в одну строку, чтобы получить доход на одного пользователя. Для этог воспользуемся сводной таблицей.

In [2]:
df = pd.pivot_table(df, index='USER_ID', columns='VARIANT_NAME',
                    values='REVENUE', aggfunc=np.sum)
df.describe()


VARIANT_NAME,control,variant
count,3931.000000,3934.000000
mean,0.163572,0.089342
std,3.385721,1.484566
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,196.010000,58.630000


In [3]:
df[(df.control.notnull()) & (df.variant.notnull())].describe()


VARIANT_NAME,control,variant
count,1541.000000,1541.000000
mean,0.111901,0.111713
std,1.498332,2.117499
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,41.530000,58.630000


In [4]:
df[(df.control.notnull()) & (df.variant.notnull())].head(10)


VARIANT_NAME,control,variant
USER_ID,,
3,0.0,0.0
10,0.0,0.0
18,0.0,0.0
25,0.0,0.0
40,0.0,0.0
57,0.0,0.0
60,0.0,0.0
64,0.0,0.0
67,0.0,0.0


Траффик между группами разделен плохо. 1541 пользователей попали в обе группы. Это 39,2 % пользователей обеих групп. Удаление такого количества пользователей (более 5%) может повлиять на результаты эксперимента.
Однако, не смотря на это, попробуем произвести оценку оставшейся выборки.

In [5]:
data_a = df[(df.control.notnull()) & (df.variant.isnull())]['control']
data_b = df[(df.control.isnull()) & (df.variant.notnull())]['variant']


Выборка A (контрольная)

In [6]:
data_a.describe()


count    2390.000000
mean        0.196887
std         4.172201
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max       196.010000
Name: control, dtype: float64

Выборка B (с изменениями на сайте)

In [7]:
data_b.describe()


count    2393.000000
mean        0.074935
std         0.858207
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max        23.040000
Name: variant, dtype: float64

Группы распределены равномерно. Средний чек и максимум в контрольной группе выше, стандартное отклонение больше. Проверку статистической значимости будем проверять с помощью t-теста, т. к. у нас имеется относительно большая выборка. Метрика распределяется не нормально.

In [8]:
stats.ttest_ind(a=data_a, b=data_b)


TtestResult(statistic=1.400504887136015, pvalue=0.16142707540780138, df=4781.0)

# Рекомендации
1. A/B-тестирование выполнено ошибками в планировании. Нарушено сплитование траффика. Более 1,5 тысяч пользователей оказались как в двух группах. Это 39% пользователей каждой группы.
2. В таких условиях корректное проведение A/B тестирование не представляется возможным.
3. Рекомендуется настроить систему разделения траффика, провести A/A тестирование и только после этого продолжать A/B-тестирования изменений. Использование существующей системы разделения траффика для этого не годится.
4. Статистически значимых различий между очищенными группами не обнаружено (p-value = 0.1614).